# Further data cleaning

In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import time

In [2]:
# Loading dataset and dropping the first column, because it contains rownumbers
df_init = pd.read_csv('data/Kickstarter_init_cleaned2.csv')
df_init2 = df_init.drop(columns='Unnamed: 0')

In [3]:
# Display numbers with two digits
pd.options.display.float_format = '{:,.2f}'.format

### Selecting data

In [4]:
# Counting unique values in column state.
df_init2.state.value_counts()

successful    117465
failed         75199
canceled        8624
live            7311
suspended        623
Name: state, dtype: int64

In [5]:
# Drop the rows of the states canceled, suspended and live
df_st = df_init2.drop(df_init2[(df_init2.state == "canceled") | (df_init2.state == "suspended")| (df_init2.state == "live")].index)

In [6]:
# Counting unique values in column state.
df_st.state.value_counts()

successful    117465
failed         75199
Name: state, dtype: int64

In [7]:
# Checking column names
df_st.columns

Index(['backers_count', 'blurb', 'converted_pledged_amount', 'country',
       'created_at', 'currency', 'currency_trailing_code', 'current_currency',
       'deadline', 'disable_communication', 'fx_rate', 'goal', 'id',
       'is_starrable', 'launched_at', 'name', 'pledged', 'slug', 'spotlight',
       'staff_pick', 'state', 'state_changed_at', 'static_usd_rate',
       'usd_pledged', 'usd_type', 'category_name', 'category_id',
       'category_parent_id', 'creator_name', 'location_name', 'location_state',
       'created_at_rd', 'deadline_rd', 'launched_at_rd',
       'state_changed_at_rd'],
      dtype='object')

In [8]:
# Drop useless columns
df_sel = df_st.drop(columns=['currency_trailing_code', 'current_currency', 'disable_communication',
                              'is_starrable','spotlight','staff_pick', 'static_usd_rate', 'usd_type'])

### Removing doubles based on id

In [9]:
# Check for duplicates in the database based on id
print(df_sel.shape[0]-df_sel.id.nunique())
double = df_sel[df_sel.duplicated(subset=['id'], keep=False)]
#double.sort_values("id")

23685


In [10]:
# Remove duplicates in the database based on id
df_id = df_sel.drop_duplicates(subset=['id'], keep="last")
#df_id[df_id.id==39036]
df_id.shape

(168979, 27)

### Removing columns with comparable content regarding the amount pledged

In [11]:
# Check the difference between converted_pledged_amount, pledged and usd_pledged
#pledged = df_id[["converted_pledged_amount","pledged","usd_pledged"]]
#pledged.describe()
#pledged.head(20)

**Pledged** is in the original currency.
**Converted_pledged_amount** is rounded.
**usd_pledged** is not rounded.
**usd_pledged** is kept.

In [12]:
# Drop columns converted_pledged_amount and pledged.
df_mis = df_id.drop(columns=["converted_pledged_amount", "pledged"])

### Check for missing data

In [13]:
#df_mis.info()

In [14]:
df1 = df_mis[df_mis.isna().any(axis=1)]
df1.shape

(8989, 25)

There are 8989 missing data values in the columns **blurb, creator_name, location_name, and location_state**.

In [15]:
# check for missing values in the blurb column
df_mis[df_mis['blurb'].isnull()]

,backers_count,blurb,country,created_at,currency,deadline,fx_rate,goal,id,launched_at,...,category_name,category_id,category_parent_id,creator_name,location_name,location_state,created_at_rd,deadline_rd,launched_at_rd,state_changed_at_rd
65168,39,NaN,DE,1504364375,EUR,1507625188,1.13,"15,000.00",937524480,1505033188,...,Ready-to-wear,269,9.00,Annabelle Deisler,Munich,Bavaria,Sat Sep 2 16:59:35 2017,Tue Oct 10 10:46:28 2017,Sun Sep 10 10:46:28 2017,Tue Oct 10 10:46:30 2017
108662,0,NaN,US,1509679461,USD,1515800048,1.00,"40,000.00",1077399482,1510616048,...,Digital Art,21,1.00,moe,Los Angeles,CA,Fri Nov 3 04:24:21 2017,Sat Jan 13 00:34:08 2018,Tue Nov 14 00:34:08 2017,Sat Jan 13 00:34:08 2018


In [16]:
# replace missing values in the column blurb with the project name.
df_mis.loc[df_mis['blurb'].isnull(),'blurb'] = df_mis['name']
#df_mis.blurb[65168]

Replace missing values in:

* **creator** name with "John Doe"
* **location state** with the country
* **location name** with the location state
* **category parent id** with the next free category number (27).

In [17]:
# replace missing values with other values.
df_mis.loc[df_mis['creator_name'].isnull(),'creator_name'] = "John Doe"
df_mis.loc[df_mis['location_state'].isnull(),'location_state'] = df_mis['country']
df_mis.loc[df_mis['location_name'].isnull(),'location_name'] = df_mis['location_state']
df_mis.loc[df_mis['category_parent_id'].isnull(),'category_parent_id'] = df_mis.category_parent_id.max()+1

### Feature engineering

In [18]:
# Goal in USD
df_mis["usd_goal"] = df_mis.goal * df_mis.fx_rate
# Remove column with exchange rate
df_feat1 = df_mis.drop(columns = ["goal","currency","fx_rate"])

In [19]:
# Project duration in days between launch and deadline
df_feat1["duration_days"] = round((df_feat1.deadline - df_feat1.launched_at)/(60*60*24))
df_feat1["duration_days_prep"] = round((df_feat1.launched_at - df_feat1.created_at)/(60*60*24))

# Remove columns with unix time stamps.
df_feat2 = df_feat1.drop(columns=["created_at", "deadline", "launched_at", "state_changed_at"])

In [20]:
# Year, month and weekday
df_feat2["year_deadline"] = df_feat2['deadline_rd'].astype(str).str[-4:]
df_feat2["month_deadline"] = df_feat2['deadline_rd'].astype(str).str[4:7]
df_feat2["weekday_deadline"] = df_feat2['deadline_rd'].astype(str).str[:3]
df_feat2["weekday_launched_at"] = df_feat2['launched_at_rd'].astype(str).str[:3]

# Which season is the month in?
winter = ["Dec","Jan","Feb"]
spring = ["Mar","Apr","May"]
summer = ["Jun","Jul","Aug"]

df_feat2['winter_deadline'] = np.where(df_feat2['month_deadline'].isin(winter), True, False)
df_feat2['spring_deadline'] = np.where(df_feat2['month_deadline'].isin(spring), True, False)
df_feat2['summer_deadline'] = np.where(df_feat2['month_deadline'].isin(summer), True, False)

# Is the weekday on the weekend?
weekend = ["Sat","Sun"]

df_feat2['deadline_weekend'] = np.where(df_feat2['weekday_deadline'].isin(weekend), True, False)
df_feat2['launched_weekend'] = np.where(df_feat2['weekday_launched_at'].isin(weekend), True, False)

# Create dummies for the five relevant columns.
df_feat3 = pd.get_dummies(df_feat2, columns=["winter_deadline","spring_deadline",
                                             "summer_deadline","deadline_weekend",
                                             "launched_weekend"],
                          drop_first=True)
# Drop superfluous columns
df_feat3 = df_feat3.drop(columns=["created_at_rd","deadline_rd","launched_at_rd","state_changed_at_rd"])

In [21]:
# Change country to a boolean operator for US and not US.
df_feat3['country_US'] = np.where(df_feat3['country'] == 'US', True, False)

# Create dummies
df_feat4 = pd.get_dummies(df_feat3, columns=['country_US'], drop_first=True)

# Drop column country.
df_feat4 = df_feat4.drop(columns="country")

In [22]:
# Change location_state to a boolean operator in two columns for eastcoast
eastern = ["ME","NH","VT","NY","MA","RI","CT","NJ","PA","DE","MD","DC","MI","OH","IN",
           "IL","WI","WV","VA","NC","TN","KY","SC","GA","AL","MS","FL"]
df_feat4['eastcoast'] = np.where(df_feat4['location_state'].isin(eastern), True, False)

# Create dummies
df_feat5 = pd.get_dummies(df_feat4, columns=['eastcoast'], drop_first=True)

# Drop columns
df_feat5 = df_feat5.drop(columns=["location_name","location_state"])

In [23]:
# Change blurb to a boolean operator for long or short blurb based on word count.
df_feat5['blurb_nwords'] = df_feat5['blurb'].str.count(' ') + 1
bmean = df_feat5.blurb_nwords.mean()
print(bmean)
df_feat5['long_blurb'] = np.where(df_feat5['blurb_nwords'] >= bmean, True, False)

# Create dummies
df_feat6 = pd.get_dummies(df_feat5, columns=['long_blurb'], drop_first=True)

# Drop columns
df_feat6 = df_feat6.drop(columns=['blurb','blurb_nwords'])

19.06536906952935


In [24]:
# Change name to a boolean operator for long or short project name based on word count.
df_feat6['name_nwords'] = df_feat6['name'].str.count(' ') + 1
bmean = df_feat6.name_nwords.mean()
print(bmean)
df_feat6['long_name'] = np.where(df_feat6['name_nwords'] >= bmean, True, False)

# Create dummies
df_feat7 = pd.get_dummies(df_feat6, columns=['long_name'], drop_first=True)

# Drop columns
df_feat7 = df_feat7.drop(columns=['name','name_nwords','slug'])

5.722456636623486


In [25]:
# Change state to a boolean operator for successful and failed.
df_feat7['state_b'] = np.where(df_feat7['state'] == 'successful', True, False)

# Create dummies
df_feat8 = pd.get_dummies(df_feat7, columns=['state_b'], drop_first=True)

# Drop columns
df_feat8 = df_feat8.drop(columns=["state"])

In [26]:
# Change creator name to a boolean operator for long or short creator name based on word count.
df_feat8['creator_name_nwords'] = df_feat8['creator_name'].str.count(' ') + 1
df_feat8['long_creator_name'] = np.where(df_feat8['creator_name_nwords'] > 2, True, False)

# Create dummies
df_feat9 = pd.get_dummies(df_feat8, columns=['long_creator_name'], drop_first=True)

# Drop columns
df_feat9 = df_feat9.drop(columns=['creator_name','creator_name_nwords'])

### Dealing with outliers of numeric features

In [27]:
df_feat9.columns

Index(['backers_count', 'id', 'usd_pledged', 'category_name', 'category_id',
       'category_parent_id', 'usd_goal', 'duration_days', 'duration_days_prep',
       'year_deadline', 'month_deadline', 'weekday_deadline',
       'weekday_launched_at', 'winter_deadline_True', 'spring_deadline_True',
       'summer_deadline_True', 'deadline_weekend_True',
       'launched_weekend_True', 'country_US_True', 'eastcoast_True',
       'long_blurb_True', 'long_name_True', 'state_b_True',
       'long_creator_name_True'],
      dtype='object')

In [28]:
# usd_goal: min is 100 USD, max is successfully pledged project with highest goal.
highest_goal = df_feat9[df_feat9.state_b_True == 1].usd_goal.max()
df = df_feat9.loc[((df_feat9.usd_goal >= 100.0) & (df_feat9.usd_goal <= highest_goal))]
df.usd_goal.describe()

count     166,192.00
mean       19,490.07
std        78,516.05
min           100.00
25%         1,502.45
50%         5,000.00
75%        13,256.80
max     2,000,000.00
Name: usd_goal, dtype: float64

In [29]:
# Check data loss
df_feat9.shape[0]-df.shape[0]

2787

In [30]:
# check for more outliers
df.describe()

,backers_count,id,usd_pledged,category_id,category_parent_id,usd_goal,duration_days,duration_days_prep,winter_deadline_True,spring_deadline_True,summer_deadline_True,deadline_weekend_True,launched_weekend_True,country_US_True,eastcoast_True,long_blurb_True,long_name_True,state_b_True,long_creator_name_True
count,"166,192.00","166,192.00","166,192.00","166,192.00","166,192.00","166,192.00","166,192.00","166,192.00","166,192.00","166,192.00","166,192.00","166,192.00","166,192.00","166,192.00","166,192.00","166,192.00","166,192.00","166,192.00","166,192.00"
mean,141.99,"1,073,397,001.68","12,297.14",171.83,12.51,"19,490.07",32.72,47.01,0.22,0.26,0.27,0.30,0.15,0.73,0.37,0.54,0.51,0.56,0.24
std,901.95,"619,431,950.59","84,403.00",134.30,6.28,"78,516.05",11.69,127.88,0.41,0.44,0.44,0.46,0.35,0.45,0.48,0.50,0.50,0.50,0.43
min,0.00,"8,624.00",0.00,1.00,1.00,100.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,3.00,"535,785,761.75",101.69,37.00,10.00,"1,502.45",30.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,26.00,"1,074,926,261.00","1,524.22",250.00,12.00,"5,000.00",30.00,11.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,1.00,1.00,0.00
75%,86.00,"1,608,503,247.50","6,372.99",300.00,16.00,"13,256.80",34.00,36.00,0.00,1.00,1.00,1.00,0.00,1.00,1.00,1.00,1.00,1.00,0.00
max,"105,857.00","2,147,476,221.00","8,596,474.58",389.00,27.00,"2,000,000.00",93.00,"3,304.00",1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [31]:
# One day seems short for a project duration.
check1 = df[df.duration_days == 1]
#check1
# Some projects were successful after one day, so we can keep it.

In [32]:
# More than 2000 days of preparation time between creating and launching the project seems large.
check2 = df[df.duration_days_prep > 2000]
#check2
# Some of these projects were movies, for which much preparation time is needed.

In [40]:
print("original_set: ",df_init.shape[0])
print("trimmed_set: ",df.shape[0])
print("difference ((original-trimmed)/original): ",(df_init.shape[0]-df.shape[0])/df_init.shape[0])

original_set:  209222
trimmed_set:  166192
difference ((original-trimmed)/original):  0.20566670808997142


In [33]:
# Save the cleaned dataset
df.to_csv('data/Kickstarter_cleaned2.csv')